In [1]:
from bs4 import BeautifulSoup
import requests
import numpy as np
import pandas as pd

### Get information from Wiki

In [2]:
source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text   
soup = BeautifulSoup(source, 'html.parser')

In [3]:
PostalCodeList = []
BoroughList = []
NeighborhoodList = []
tbody = soup.find('tbody')

### Collect the Data

In [4]:
for index, value in enumerate(tbody.find_all('td')):
    if (index%3 == 0):
        PostalCodeList.append(value.text.strip())
    elif(index%3 == 1):
        BoroughList.append(value.text.strip())
    else:
        NeighborhoodList.append(value.text.strip())
dataDic = { "PostalCode":PostalCodeList, "Borough":BoroughList, "Neighborhood": NeighborhoodList }

### Translate to DataFrame

In [5]:
df = pd.DataFrame.from_dict(dataDic)
print( df.head() )

            Borough      Neighborhood PostalCode
0      Not assigned      Not assigned        M1A
1      Not assigned      Not assigned        M2A
2        North York         Parkwoods        M3A
3        North York  Victoria Village        M4A
4  Downtown Toronto      Harbourfront        M5A


### Remove column if Boroug is "Not assigned"

In [6]:
df = df[df.Borough != 'Not assigned']
df.reset_index(drop=True, inplace=True)
print( df.head() )

            Borough      Neighborhood PostalCode
0        North York         Parkwoods        M3A
1        North York  Victoria Village        M4A
2  Downtown Toronto      Harbourfront        M5A
3  Downtown Toronto       Regent Park        M5A
4        North York  Lawrence Heights        M6A


In [13]:
aggregate_fun = {'PostalCode': 'first',
                 "Borough": 'first', 
                 "Neighborhood": lambda col: ','.join(col)}
df_new = df.groupby(df['PostalCode']).aggregate(aggregate_fun)

df_new.reset_index(drop=True, inplace=True)
#df_new

,Borough,Neighborhood,PostalCode
0,Scarborough,"Rouge,Malvern",M1B
1,Scarborough,"Highland Creek,Rouge Hill,Port Union",M1C
2,Scarborough,"Guildwood,Morningside,West Hill",M1E
3,Scarborough,Woburn,M1G
4,Scarborough,Cedarbrae,M1H
5,Scarborough,Scarborough Village,M1J
6,Scarborough,"East Birchmount Park,Ionview,Kennedy Park",M1K
7,Scarborough,"Clairlea,Golden Mile,Oakridge",M1L
8,Scarborough,"Cliffcrest,Cliffside,Scarborough Village West",M1M
9,Scarborough,"Birch Cliff,Cliffside West",M1N


In [14]:
for index, row in df_new.iterrows():
    if (row.Neighborhood) == 'Not assigned':
        row.Neighborhood = row.Borough


In [15]:
print(df_new.iloc[85])

Borough         Queen's Park
Neighborhood    Queen's Park
PostalCode               M7A
Name: 85, dtype: object


### The shape of List of postal codes of Canada

In [18]:
df_new.shape

(103, 3)